<a href="https://colab.research.google.com/github/kritisinghh/Trading/blob/main/RL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Run this in a NEW Colab notebook cell.
from google.colab import drive
import os, time
from pathlib import Path
import nbformat

# 1) remount Drive
drive.mount('/content/drive', force_remount=True)

# 2) walk MyDrive and find all .ipynb files
root = Path('/content/drive/MyDrive')
notebooks = []
for dirpath, dirnames, filenames in os.walk(root):
    for fn in filenames:
        if fn.endswith('.ipynb'):
            p = Path(dirpath) / fn
            try:
                mtime = time.ctime(p.stat().st_mtime)
                size = p.stat().st_size
            except Exception:
                mtime = "n/a"; size = -1
            notebooks.append((str(p), size, mtime))

# 3) print results
if not notebooks:
    print("No .ipynb files found under /content/drive/MyDrive. Check other folders or move your file into MyDrive.")
else:
    print(f"Found {len(notebooks)} .ipynb files. Showing first 200 matches (if available):\n")
    for i, (path, size, mtime) in enumerate(notebooks[:200], 1):
        print(f"{i:3d}. {path} — {size} bytes — modified: {mtime}")

# 4) Choose which notebook to clean:
# By default this will clean the FIRST notebook found.
# If you want a different one, copy the exact path from the printed list and set TARGET_PATH below.
if notebooks:
    TARGET_PATH = notebooks[0][0]   # <<-- change this to an exact string if you prefer another file
    print("\nDefault target to clean (first match):", TARGET_PATH)

    # 5) Load and clean notebook (remove widgets, outputs, normalize)
    nb = nbformat.read(TARGET_PATH, as_version=4)
    changed = False
    if 'widgets' in nb.metadata:
        del nb.metadata['widgets']; changed = True
    nb.metadata.setdefault('kernelspec', nb.metadata.get('kernelspec', {}))
    nb.nbformat = 4
    nb.nbformat_minor = max(0, int(getattr(nb, 'nbformat_minor', 5)))
    for cell in nb.cells:
        if 'widgets' in cell.get('metadata', {}):
            del cell['metadata']['widgets']; changed = True
        if 'application/vnd.jupyter.widget-state+json' in cell.get('metadata', {}):
            cell['metadata'].pop('application/vnd.jupyter.widget-state+json', None); changed = True
        if 'outputs' in cell and cell['outputs']:
            cell['outputs'] = []; changed = True
        if 'execution_count' in cell and cell['execution_count'] is not None:
            cell['execution_count'] = None; changed = True

    clean_path = Path(TARGET_PATH).with_name(Path(TARGET_PATH).stem + '_clean.ipynb')
    nbformat.write(nb, str(clean_path))
    print("\nSaved cleaned notebook to:", clean_path)
    print("Changed metadata/outputs?:", changed)
    print("\nIf you want a different notebook cleaned, set TARGET_PATH to one of the printed paths and rerun the cell.")


Mounted at /content/drive
Found 82 .ipynb files. Showing first 200 matches (if available):

  1. /content/drive/MyDrive/Colab Notebooks/Kriti.ipynb — 28839 bytes — modified: Wed Jun  7 08:58:38 2023
  2. /content/drive/MyDrive/Colab Notebooks/Copy of Kriti (1).ipynb — 28912 bytes — modified: Wed Jun  7 08:59:19 2023
  3. /content/drive/MyDrive/Colab Notebooks/Copy of R Programming (1).ipynb — 1320152 bytes — modified: Fri Jun  9 11:40:21 2023
  4. /content/drive/MyDrive/Colab Notebooks/Copy of topsis (1).ipynb — 13098 bytes — modified: Fri Jun  9 11:58:23 2023
  5. /content/drive/MyDrive/Colab Notebooks/Python.ipynb — 85063 bytes — modified: Fri Jun  9 12:38:51 2023
  6. /content/drive/MyDrive/Colab Notebooks/Object Oriented Programming .ipynb — 38719 bytes — modified: Wed Jun 21 10:28:14 2023
  7. /content/drive/MyDrive/Colab Notebooks/Working with Pandas.ipynb — 359068 bytes — modified: Wed Jun 21 10:37:08 2023
  8. /content/drive/MyDrive/Colab Notebooks/Copy of Working_with_Pandas (

In [12]:
# (Optional) Install packages if running in Colab
# Uncomment to run in Colab if packages missing.
# !pip install --quiet yfinance pandas numpy matplotlib ipywidgets ta stable-baselines3==2.0.0


In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
%matplotlib inline


In [14]:
def download_data(ticker, start='2018-01-01', end='2024-12-31'):
    df = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        try:
            lvl0 = list(df.columns.get_level_values(0))
            lvl1 = list(df.columns.get_level_values(1))
            if len(set(lvl1)) == 1:
                df.columns = lvl0
        except Exception:
            df.columns = [c[0] if isinstance(c, tuple) else c for c in df.columns]
    df = df.dropna()
    return df

def add_basic_indicators(df):
    df = df.copy()
    df['return_1'] = df['Close'].pct_change()
    df['sma_5'] = df['Close'].rolling(5).mean()
    df['sma_20'] = df['Close'].rolling(20).mean()
    df = df.dropna().reset_index()
    return df

ticker = 'AAPL'
df_raw = download_data(ticker, start='2018-01-01', end='2024-12-31')
df = add_basic_indicators(df_raw)
print('Loaded', ticker, 'shape:', df.shape)
df.head()

Loaded AAPL shape: (1741, 9)


/tmp/ipython-input-2371377968.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, progress=False)


,Date,Close,High,Low,Open,Volume,return_1,sma_5,sma_20
0,2018-01-30,39.103012,39.196687,38.571396,38.765775,184192800,-0.005894,39.895522,40.827603
1,2018-01-31,39.210732,39.447268,38.992935,39.079585,129915600,0.002755,39.577486,40.771045
2,2018-02-01,39.292713,39.489433,39.053836,39.149856,188923200,0.002091,39.421514,40.718938
3,2018-02-02,37.587784,39.063193,37.494109,38.875839,346375200,-0.043390,38.905822,40.572216
4,2018-02-05,36.648689,38.379367,36.533934,37.259930,290954000,-0.024984,38.368586,40.355472


In [15]:
class ManualTrader:
    def __init__(self, df, initial_cash=10000, transaction_cost_pct=0.001):
        self.df = df.reset_index(drop=True)
        self.initial_cash = initial_cash
        self.transaction_cost_pct = transaction_cost_pct
        self.reset()
        self.btn_buy = widgets.Button(description='Buy', button_style='success')
        self.btn_hold = widgets.Button(description='Hold')
        self.btn_sell = widgets.Button(description='Sell', button_style='danger')
        self.btn_next = widgets.Button(description='Auto-advance 10 steps', button_style='info')
        self.out = widgets.Output(layout={'border': '1px solid black'})
        self.btn_buy.on_click(self.on_buy)
        self.btn_hold.on_click(self.on_hold)
        self.btn_sell.on_click(self.on_sell)
        self.btn_next.on_click(self.on_advance10)
        self.controls = widgets.HBox([self.btn_buy, self.btn_hold, self.btn_sell, self.btn_next])
        display(self.controls, self.out)
        self.render()

    def reset(self):
        self.current_step = 0 + 30
        self.cash = float(self.initial_cash)
        self.position = 0
        self.position_price = 0.0
        self.portfolio_history = []
        self.trade_log = []

    def get_price(self, step=None):
        if step is None:
            step = self.current_step
        return float(self.df.loc[step, 'Close'])

    def portfolio_value(self):
        return self.cash + self.position * self.get_price()

    def step_action(self, action):
        price = self.get_price()
        prev_value = self.portfolio_value()
        if action == 'buy' and self.position == 0:
            cost = price * (1 + self.transaction_cost_pct)
            if self.cash >= cost:
                self.cash -= cost
                self.position = 1
                self.position_price = price
                self.trade_log.append({'step': self.current_step, 'action': 'buy', 'price': price})
        elif action == 'sell' and self.position == 1:
            proceeds = price * (1 - self.transaction_cost_pct)
            self.cash += proceeds
            self.position = 0
            self.trade_log.append({'step': self.current_step, 'action': 'sell', 'price': price})
        self.current_step += 1
        value = self.portfolio_value()
        self.portfolio_history.append(value)
        return value - prev_value

    def on_buy(self, _):
        self._handle_action('buy')

    def on_hold(self, _):
        self._handle_action('hold')

    def on_sell(self, _):
        self._handle_action('sell')

    def on_advance10(self, _):
        for _ in range(10):
            if self.current_step >= len(self.df)-1:
                break
            self._handle_action('hold')

    def _handle_action(self, action):
        if self.current_step >= len(self.df)-1:
            with self.out:
                print('End of data reached. Resetting...')
            return
        reward = self.step_action(action)
        self.render(action, reward)

    def render(self, last_action=None, reward=0.0):
        with self.out:
            clear_output(wait=True)
            step = self.current_step
            price = self.get_price(step-1)
            fig, ax = plt.subplots(2,1, figsize=(10,6), gridspec_kw={'height_ratios':[2,1]})
            ax[0].plot(self.df['Date'][:step], self.df['Close'][:step], label='Close')
            ax[0].plot(self.df['Date'][:step], self.df['sma_5'][:step], label='SMA5', alpha=0.7)
            ax[0].plot(self.df['Date'][:step], self.df['sma_20'][:step], label='SMA20', alpha=0.7)
            ax[0].legend(loc='upper left')
            ax[0].set_title(f'Time {step-1} | Price: {price:.2f} | Last action: {last_action} | Reward: {reward:.2f}')
            if len(self.portfolio_history)>0:
                ax[1].plot(self.portfolio_history, label='Portfolio Value')
            ax[1].axhline(self.initial_cash, color='gray', linestyle='--', label='Initial Cash')
            ax[1].legend(loc='upper left')
            plt.tight_layout()
            display(fig)
            print(f"Step: {step-1}  |  Price: {price:.2f}  |  Cash: {self.cash:.2f}  |  Position: {self.position}  |  Portfolio: {self.portfolio_value():.2f}")
            print('Recent trades (last 10):')
            for t in self.trade_log[-10:]:
                print(t)
            plt.close(fig)

trader = ManualTrader(df=df, initial_cash=10000)


Output(layout=Layout(border='1px solid black'))

In [16]:
try:
    from stable_baselines3 import PPO
    from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
    import gym
    class SimpleGymEnv(gym.Env):
        def __init__(self, df, window_size=30, initial_cash=10000, transaction_cost_pct=0.001):
            super().__init__()
            self.df = df.reset_index(drop=True)
            self.window_size = window_size
            self.transaction_cost_pct = transaction_cost_pct
            self.initial_cash = initial_cash
            self.feature_cols = ['Close']
            obs_dim = window_size * len(self.feature_cols) + 3
            self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32)
            self.action_space = gym.spaces.Discrete(3)
            self.reset()
        def _get_obs(self):
            start = self.current_step - self.window_size + 1
            window = self.df.loc[start:self.current_step, self.feature_cols].values
            flat = window.flatten()
            cash_norm = np.array([self.cash / self.initial_cash], dtype=np.float32)
            position = np.array([self.position], dtype=np.float32)
            position_price_norm = np.array([0.0], dtype=np.float32)
            if self.position == 1:
                position_price_norm = np.array([self.position_price / self.initial_cash], dtype=np.float32)
            return np.concatenate([flat, cash_norm, position, position_price_norm]).astype(np.float32)
        def reset(self):
            self.current_step = self.window_size - 1
            self.cash = self.initial_cash
            self.position = 0
            self.position_price = 0.0
            self.total_trades = 0
            return self._get_obs()
        def step(self, action):
            price = float(self.df.loc[self.current_step,'Close'])
            prev_portfolio = self.cash + self.position*price
            if action==1 and self.position==0:
                cost = price*(1+self.transaction_cost_pct)
                if self.cash>=cost:
                    self.cash-=cost; self.position=1; self.position_price=price
            elif action==2 and self.position==1:
                proceeds=price*(1-self.transaction_cost_pct)
                self.cash+=proceeds; self.position=0; self.position_price=0.0
            self.current_step+=1
            done = self.current_step>=len(self.df)-1
            current_price = float(self.df.loc[self.current_step,'Close']) if not done else price
            portfolio = self.cash + self.position*current_price
            reward = portfolio - prev_portfolio
            obs = self._get_obs() if not done else np.zeros(self.observation_space.shape)
            info = {'portfolio_value': portfolio}
            return obs, float(reward), done, info

    train_env = DummyVecEnv([lambda: SimpleGymEnv(df=df, window_size=30)])
    train_env = VecNormalize(train_env, norm_obs=True, norm_reward=False, clip_obs=10.)
    model = PPO('MlpPolicy', train_env, verbose=1)
    TOTAL = 10000
    model.learn(total_timesteps=TOTAL)
    model.save('/content/ppo_trader_demo')
    print('Saved model to /content/ppo_trader_demo')
except Exception as e:
    print('Skipping PPO demo (missing packages or runtime issue):', e)


Skipping PPO demo (missing packages or runtime issue): Missing shimmy installation. You an OpenAI Gym environment. Stable-Baselines3 (SB3) has transitioned to using Gymnasium internally. In order to use OpenAI Gym environments with SB3, you need to install shimmy (`pip install 'shimmy>=0.2.1'`).


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [7]:
!pip install stable-baselines3==2.0.0 gym==0.26.2
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 46.6 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827727 sha256=1617a7c95e5355fc120ea58694d6ffe89f4406b67d68bdc1bd4b5c3a1aa11942
  Stored in directory: /root/.cache/pip/wheels/95/51/6c/9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.2.2
    Uninstalling gymnasium-1.2.2:
      Successfully uninstalled gymnasium-1.2.2
ERROR: pip's dependency resolver does n